In [1]:
from numpy import *

In [2]:
def loadDataSet():

    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
def createC1(dataSet):

    """createC1（创建集合 C1）



    Args:

        dataSet 原始数据集

    Returns:

        frozenset 返回一个 frozenset 格式的 list

    """



    C1 = []

    for transaction in dataSet:

        for item in transaction:

            if not [item] in C1:

                # 遍历所有的元素，如果不在 C1 出现过，那么就 append

                C1.append([item])

    # 对数组进行 `从小到大` 的排序

    # print 'sort 前=', C1

    C1.sort()

    # frozenset 表示冻结的 set 集合，元素无改变；可以把它当字典的 key 来使用

    # print 'sort 后=', C1

    # print 'frozenset=', map(frozenset, C1)

    return map(frozenset, C1)

In [4]:
def scanD(D, Ck, minSupport):
    """scanD（计算候选数据集 CK 在数据集 D 中的支持度，并返回支持度大于最小支持度 minSupport 的数据）



    Args:

        D 数据集

        Ck 候选项集列表

        minSupport 最小支持度

    Returns:

        retList 支持度大于 minSupport 的集合

        supportData 候选项集支持度数据

    """



    # ssCnt 临时存放选数据集 Ck 的频率. 例如: a->10, b->5, c->8    

    ssCnt = {}

    for tid in D:

        for can in Ck:

            # s.issubset(t)  测试是否 s 中的每一个元素都在 t 中

            if can.issubset(tid):

                if can not in ssCnt:

                    ssCnt[can] = 1

                else:

                    ssCnt[can] += 1

    numItems = float(len(D)) # 数据集 D 的数量

    retList = []

    supportData = {}
    print(ssCnt)

    for key in ssCnt:

        # 支持度 = 候选项（key）出现的次数 / 所有数据集的数量

        support = ssCnt[key]/numItems

        if support >= minSupport:

            # 在 retList 的首位插入元素，只存储支持度满足频繁项集的值

            retList.insert(0, key)

        # 存储所有的候选项（key）和对应的支持度（support）

        supportData[key] = support

    return retList, supportData



In [5]:
dataSet=loadDataSet()

In [6]:
C1=createC1(dataSet)

In [7]:
D=map(set,dataSet)

In [8]:
L1,suppData0=scanD(list(D),list(C1),0.5)

{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}


In [9]:
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [10]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

In [11]:
def aprioriGen(Lk, k):

    """aprioriGen（输入频繁项集列表 Lk 与返回的元素个数 k，然后输出候选项集 Ck。

       例如: 以 {0},{1},{2} 为输入且 k = 2 则输出 {0,1}, {0,2}, {1,2}. 以 {0,1},{0,2},{1,2} 为输入且 k = 3 则输出 {0,1,2}

       仅需要计算一次，不需要将所有的结果计算出来，然后进行去重操作

       这是一个更高效的算法）



    Args:

        Lk 频繁项集列表

        k 返回的项集元素个数（若元素的前 k-2 相同，就进行合并）

    Returns:

        retList 元素两两合并的数据集

    """

    

    retList = []

    lenLk = len(Lk)

    for i in range(lenLk):

        for j in range(i+1, lenLk):

            L1 = list(Lk[i])[: k-2]

            L2 = list(Lk[j])[: k-2]

            # print '-----i=', i, k-2, Lk, Lk[i], list(Lk[i])[: k-2]

            # print '-----j=', j, k-2, Lk, Lk[j], list(Lk[j])[: k-2]

            L1.sort()

            L2.sort()

            # 第一次 L1,L2 为空，元素直接进行合并，返回元素两两合并的数据集

            # if first k-2 elements are equal

            if L1 == L2:

                # set union

                # print 'union=', Lk[i] | Lk[j], Lk[i], Lk[j]

                retList.append(Lk[i] | Lk[j])

    return retList

In [12]:
def apriori(dataSet, minSupport=0.5):

    """apriori（首先构建集合 C1，然后扫描数据集来判断这些只有一个元素的项集是否满足最小支持度的要求。那么满足最小支持度要求的项集构成集合 L1。然后 L1 中的元素相互组合成 C2，C2 再进一步过滤变成 L2，然后以此类推，知道 CN 的长度为 0 时结束，即可找出所有频繁项集的支持度。）



    Args:

        dataSet 原始数据集

        minSupport 支持度的阈值

    Returns:

        L 频繁项集的全集

        supportData 所有元素和支持度的全集

    """

    # C1 即对 dataSet 进行去重，排序，放入 list 中，然后转换所有的元素为 frozenset

    C1 = createC1(dataSet)

    # print 'C1: ', C1

    # 对每一行进行 set 转换，然后存放到集合中

    D = map(set, dataSet)

    # print 'D=', D

    # 计算候选数据集 C1 在数据集 D 中的支持度，并返回支持度大于 minSupport 的数据

    L1, supportData = scanD(list(D), list(C1), minSupport)

    # print "L1=", L1, "\n", "outcome: ", supportData



    # L 加了一层 list, L 一共 2 层 list

    L = [L1]

    k = 2

    # 判断 L 的第 k-2 项的数据长度是否 > 0。第一次执行时 L 为 [[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]]。L[k-2]=L[0]=[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])]，最后面 k += 1

    while (len(L[k-2]) > 0):

        # print 'k=', k, L, L[k-2]

        Ck = aprioriGen(L[k-2], k) # 例如: 以 {0},{1},{2} 为输入且 k = 2 则输出 {0,1}, {0,2}, {1,2}. 以 {0,1},{0,2},{1,2} 为输入且 k = 3 则输出 {0,1,2}

        # print 'Ck', Ck


        D = map(set, dataSet)
        Lk, supK = scanD(list(D), Ck, minSupport) # 计算候选数据集 CK 在数据集 D 中的支持度，并返回支持度大于 minSupport 的数据

        # 保存所有候选项集的支持度，如果字典没有，就追加元素，如果有，就更新元素

        supportData.update(supK)

        if len(Lk) == 0:

            break

        # Lk 表示满足频繁子项的集合，L 元素在增加，例如: 

        # l=[[set(1), set(2), set(3)]]

        # l=[[set(1), set(2), set(3)], [set(1, 2), set(2, 3)]]

        L.append(Lk)

        k += 1

        # print 'k=', k, len(L[k-2])

    return L, supportData



In [13]:
L,suppData=apriori(dataSet,0.5)

{frozenset({1}): 2, frozenset({3}): 3, frozenset({4}): 1, frozenset({2}): 3, frozenset({5}): 3}
{frozenset({1, 3}): 2, frozenset({2, 5}): 3, frozenset({3, 5}): 2, frozenset({2, 3}): 2, frozenset({1, 5}): 1, frozenset({1, 2}): 1}
{frozenset({2, 3, 5}): 2}
{}


In [14]:
dataSet=loadDataSet()

In [15]:
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [16]:
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})]]

In [17]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [18]:
def generateRules(L, supportData, minConf=0.7):

    """generateRules



    Args:

        L 频繁项集列表

        supportData 频繁项集支持度的字典

        minConf 最小置信度

    Returns:

        bigRuleList 可信度规则列表（关于 (A->B+置信度) 3个字段的组合）

    """

    bigRuleList = []

    # 假设 L = [[frozenset([1]), frozenset([3]), frozenset([2]), frozenset([5])], [frozenset([1, 3]), frozenset([2, 5]), frozenset([2, 3]), frozenset([3, 5])], [frozenset([2, 3, 5])]]

    for i in range(1, len(L)):

        # 获取频繁项集中每个组合的所有元素

        for freqSet in L[i]:

            # 假设：freqSet= frozenset([1, 3]), H1=[frozenset([1]), frozenset([3])]

            # 组合总的元素并遍历子元素，并转化为 frozenset 集合，再存放到 list 列表中

            H1 = [frozenset([item]) for item in freqSet]

            # 2 个的组合，走 else, 2 个以上的组合，走 if

            if (i > 1):

                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)

            else:

                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    return bigRuleList

In [19]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):

    """calcConf（对两个元素的频繁项，计算可信度，例如： {1,2}/{1} 或者 {1,2}/{2} 看是否满足条件）



    Args:

        freqSet 频繁项集中的元素，例如: frozenset([1, 3])    

        H 频繁项集中的元素的集合，例如: [frozenset([1]), frozenset([3])]

        supportData 所有元素的支持度的字典

        brl 关联规则列表的空数组

        minConf 最小可信度

    Returns:

        prunedH 记录 可信度大于阈值的集合

    """

    # 记录可信度大于最小可信度（minConf）的集合

    prunedH = []

    for conseq in H: # 假设 freqSet = frozenset([1, 3]), H = [frozenset([1]), frozenset([3])]，那么现在需要求出 frozenset([1]) -> frozenset([3]) 的可信度和 frozenset([3]) -> frozenset([1]) 的可信度



        # print 'confData=', freqSet, H, conseq, freqSet-conseq

        conf = supportData[freqSet]/supportData[freqSet-conseq] # 支持度定义: a -> b = support(a | b) / support(a). 假设  freqSet = frozenset([1, 3]), conseq = [frozenset([1])]，那么 frozenset([1]) 至 frozenset([3]) 的可信度为 = support(a | b) / support(a) = supportData[freqSet]/supportData[freqSet-conseq] = supportData[frozenset([1, 3])] / supportData[frozenset([1])]

        if conf >= minConf:

            # 只要买了 freqSet-conseq 集合，一定会买 conseq 集合（freqSet-conseq 集合和 conseq集合 是全集）

            print (freqSet-conseq, '-->', conseq, 'conf:', conf)

            brl.append((freqSet-conseq, conseq, conf))

            prunedH.append(conseq)

    return prunedH

In [20]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):

    """rulesFromConseq



    Args:

        freqSet 频繁项集中的元素，例如: frozenset([2, 3, 5])    

        H 频繁项集中的元素的集合，例如: [frozenset([2]), frozenset([3]), frozenset([5])]

        supportData 所有元素的支持度的字典

        brl 关联规则列表的数组

        minConf 最小可信度

    """

    # H[0] 是 freqSet 的元素组合的第一个元素，并且 H 中所有元素的长度都一样，长度由 aprioriGen(H, m+1) 这里的 m + 1 来控制

    # 该函数递归时，H[0] 的长度从 1 开始增长 1 2 3 ...

    # 假设 freqSet = frozenset([2, 3, 5]), H = [frozenset([2]), frozenset([3]), frozenset([5])]

    # 那么 m = len(H[0]) 的递归的值依次为 1 2

    # 在 m = 2 时, 跳出该递归。假设再递归一次，那么 H[0] = frozenset([2, 3, 5])，freqSet = frozenset([2, 3, 5]) ，没必要再计算 freqSet 与 H[0] 的关联规则了。

    m = len(H[0])

    if (len(freqSet) > (m + 1)):

        # print 'freqSet******************', len(freqSet), m + 1, freqSet, H, H[0]

        # 生成 m+1 个长度的所有可能的 H 中的组合，假设 H = [frozenset([2]), frozenset([3]), frozenset([5])]

        # 第一次递归调用时生成 [frozenset([2, 3]), frozenset([2, 5]), frozenset([3, 5])]

        # 第二次 。。。没有第二次，递归条件判断时已经退出了

        Hmp1 = aprioriGen(H, m+1)

        # 返回可信度大于最小可信度的集合

        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)

        print ('Hmp1=', Hmp1)

        print ('len(Hmp1)=', len(Hmp1), 'len(freqSet)=', len(freqSet))

        # 计算可信度后，还有数据大于最小可信度的话，那么继续递归调用，否则跳出递归

        if (len(Hmp1) > 1):

            # print '----------------------', Hmp1

            # print len(freqSet),  len(Hmp1[0]) + 1

            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [21]:
rules=generateRules(L,suppData,0.5)

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666
Hmp1= [frozenset({2, 3}), frozenset({2, 5}), frozenset({3, 5})]
len(Hmp1)= 3 len(freqSet)= 3


In [22]:
rules=generateRules(L,suppData,0.7)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0
Hmp1= []
len(Hmp1)= 0 len(freqSet)= 3


In [23]:
 dataSet = [line.split() for line in open("data/mushroom.dat").readlines()]

In [26]:
 L, supportData = apriori(dataSet, minSupport=0.3)

{frozenset({'1'}): 3916, frozenset({'107'}): 1248, frozenset({'113'}): 368, frozenset({'13'}): 2284, frozenset({'23'}): 3376, frozenset({'25'}): 256, frozenset({'3'}): 3656, frozenset({'34'}): 7914, frozenset({'36'}): 6812, frozenset({'38'}): 2512, frozenset({'40'}): 408, frozenset({'52'}): 3516, frozenset({'54'}): 1120, frozenset({'59'}): 5176, frozenset({'63'}): 4936, frozenset({'67'}): 4464, frozenset({'76'}): 4384, frozenset({'85'}): 8124, frozenset({'86'}): 7924, frozenset({'9'}): 2556, frozenset({'90'}): 7488, frozenset({'93'}): 3968, frozenset({'98'}): 1872, frozenset({'108'}): 400, frozenset({'114'}): 2148, frozenset({'14'}): 1072, frozenset({'2'}): 4208, frozenset({'26'}): 400, frozenset({'39'}): 5612, frozenset({'55'}): 556, frozenset({'99'}): 1968, frozenset({'115'}): 292, frozenset({'15'}): 1040, frozenset({'27'}): 400, frozenset({'4'}): 452, frozenset({'41'}): 1048, frozenset({'10'}): 3244, frozenset({'109'}): 384, frozenset({'16'}): 1840, frozenset({'24'}): 4748, frozense

{frozenset({'1', '34', '36'}): 3786, frozenset({'1', '34', '86'}): 3890, frozenset({'1', '34', '90'}): 3808, frozenset({'23', '34', '59'}): 3232, frozenset({'23', '34', '63'}): 3040, frozenset({'23', '34', '86'}): 3376, frozenset({'23', '34', '93'}): 3184, frozenset({'34', '38', '3'}): 1024, frozenset({'34', '3', '52'}): 1586, frozenset({'34', '3', '59'}): 2328, frozenset({'34', '63', '3'}): 2208, frozenset({'34', '76', '3'}): 2024, frozenset({'34', '86', '3'}): 3602, frozenset({'1', '36', '86'}): 3804, frozenset({'1', '36', '90'}): 3696, frozenset({'36', '3', '23'}): 1568, frozenset({'36', '34', '23'}): 3272, frozenset({'36', '23', '52'}): 1256, frozenset({'36', '23', '59'}): 3128, frozenset({'36', '63', '23'}): 2936, frozenset({'36', '76', '23'}): 2024, frozenset({'36', '86', '23'}): 3272, frozenset({'36', '93', '23'}): 3080, frozenset({'36', '34', '3'}): 2962, frozenset({'36', '3', '52'}): 1432, frozenset({'36', '3', '59'}): 1984, frozenset({'36', '63', '3'}): 1864, frozenset({'36',

{frozenset({'93', '86', '76', '63'}): 2128, frozenset({'93', '86', '76', '59'}): 2320, frozenset({'93', '86', '63', '59'}): 3136, frozenset({'93', '34', '86', '76'}): 2608, frozenset({'93', '34', '86', '63'}): 3280, frozenset({'93', '34', '86', '59'}): 3472, frozenset({'93', '34', '76', '63'}): 2128, frozenset({'93', '34', '76', '59'}): 2320, frozenset({'93', '34', '63', '59'}): 3136, frozenset({'93', '86', '63', '90'}): 3032, frozenset({'93', '86', '90', '59'}): 3224, frozenset({'36', '93', '86', '90'}): 3168, frozenset({'23', '93', '86', '90'}): 3080, frozenset({'85', '93', '86', '90'}): 3368, frozenset({'85', '93', '76', '90'}): 2216, frozenset({'85', '93', '63', '90'}): 3224, frozenset({'85', '93', '90', '59'}): 3416, frozenset({'85', '93', '90', '52'}): 1448, frozenset({'85', '93', '90', '38'}): 648, frozenset({'85', '93', '34', '90'}): 3368, frozenset({'85', '93', '3', '90'}): 1808, frozenset({'67', '86', '76', '90'}): 3080, frozenset({'86', '76', '63', '90'}): 2696, frozenset({'

{frozenset({'36', '34', '86', '1', '3'}): 1606, frozenset({'36', '34', '86', '1', '52'}): 1770, frozenset({'36', '34', '86', '1', '59'}): 1432, frozenset({'36', '34', '86', '63', '1'}): 1432, frozenset({'36', '34', '86', '1', '76'}): 1576, frozenset({'36', '34', '86', '1', '90'}): 3696, frozenset({'36', '34', '1', '90', '52'}): 1680, frozenset({'36', '34', '1', '90', '59'}): 1360, frozenset({'34', '86', '1', '90', '38'}): 2216, frozenset({'34', '86', '1', '90', '52'}): 1784, frozenset({'34', '86', '1', '90', '59'}): 1464, frozenset({'34', '63', '90', '59', '23'}): 2672, frozenset({'34', '86', '63', '59', '23'}): 2968, frozenset({'34', '86', '90', '59', '23'}): 2936, frozenset({'34', '63', '86', '90', '23'}): 2744, frozenset({'34', '93', '63', '59', '23'}): 2776, frozenset({'34', '93', '86', '59', '23'}): 3040, frozenset({'34', '93', '90', '59', '23'}): 2936, frozenset({'34', '93', '63', '86', '23'}): 2848, frozenset({'34', '93', '63', '90', '23'}): 2744, frozenset({'34', '93', '86', '9

{frozenset({'93', '34', '86', '63', '90', '59'}): 2960, frozenset({'93', '34', '86', '63', '59', '23'}): 2776, frozenset({'93', '63', '86', '90', '59', '23'}): 2672, frozenset({'34', '93', '86', '63', '90', '23'}): 2744, frozenset({'34', '93', '86', '90', '59', '23'}): 2936, frozenset({'34', '93', '63', '90', '59', '23'}): 2672, frozenset({'85', '93', '86', '63', '90', '59'}): 2960, frozenset({'85', '93', '86', '63', '90', '23'}): 2744, frozenset({'85', '93', '86', '90', '59', '23'}): 2936, frozenset({'85', '93', '86', '63', '59', '23'}): 2776, frozenset({'36', '93', '63', '86', '90', '59'}): 2760, frozenset({'36', '93', '63', '86', '90', '23'}): 2640, frozenset({'36', '93', '86', '90', '59', '23'}): 2832, frozenset({'36', '93', '86', '63', '59', '23'}): 2672, frozenset({'85', '63', '86', '90', '59', '76'}): 2216, frozenset({'85', '63', '86', '90', '23', '76'}): 1592, frozenset({'85', '86', '90', '59', '23', '76'}): 1784, frozenset({'85', '86', '63', '59', '23', '76'}): 1720, frozenset

{frozenset({'36', '34', '63', '86', '90', '59', '23'}): 2568, frozenset({'36', '34', '93', '63', '86', '59', '23'}): 2672, frozenset({'36', '34', '93', '63', '86', '90', '23'}): 2640, frozenset({'36', '34', '93', '63', '86', '90', '59'}): 2760, frozenset({'36', '34', '93', '63', '90', '59', '23'}): 2568, frozenset({'36', '34', '93', '86', '90', '59', '23'}): 2832, frozenset({'85', '34', '86', '63', '59', '23', '76'}): 1720, frozenset({'85', '34', '86', '90', '59', '23', '76'}): 1784, frozenset({'85', '34', '86', '63', '90', '59', '23'}): 2672, frozenset({'85', '34', '86', '63', '90', '23', '76'}): 1592, frozenset({'85', '34', '86', '63', '90', '59', '76'}): 2216, frozenset({'85', '34', '93', '63', '86', '59', '23'}): 2776, frozenset({'85', '34', '93', '63', '86', '90', '23'}): 2744, frozenset({'85', '34', '93', '63', '86', '90', '59'}): 2960, frozenset({'85', '34', '93', '63', '90', '59', '23'}): 2672, frozenset({'85', '34', '93', '86', '90', '59', '23'}): 2936, frozenset({'85', '36', 

{frozenset({'85', '36', '93', '34', '86', '63', '90', '59', '23'}): 2568, frozenset({'85', '36', '34', '39', '86', '2', '63', '59', '23'}): 2464, frozenset({'85', '36', '93', '34', '39', '86', '90', '59', '23'}): 2576, frozenset({'85', '36', '93', '34', '39', '86', '2', '59', '23'}): 2464}
{}


In [29]:
L[2]

[frozenset({'1', '110', '24'}),
 frozenset({'1', '24', '85'}),
 frozenset({'1', '24', '90'}),
 frozenset({'1', '24', '86'}),
 frozenset({'1', '24', '36'}),
 frozenset({'1', '24', '34'}),
 frozenset({'110', '36', '53'}),
 frozenset({'36', '53', '85'}),
 frozenset({'36', '56', '90'}),
 frozenset({'39', '56', '90'}),
 frozenset({'39', '56', '86'}),
 frozenset({'36', '56', '85'}),
 frozenset({'39', '56', '85'}),
 frozenset({'34', '39', '56'}),
 frozenset({'116', '36', '90'}),
 frozenset({'116', '36', '85'}),
 frozenset({'36', '56', '86'}),
 frozenset({'34', '36', '56'}),
 frozenset({'36', '39', '56'}),
 frozenset({'116', '36', '86'}),
 frozenset({'116', '34', '36'}),
 frozenset({'36', '53', '90'}),
 frozenset({'36', '53', '86'}),
 frozenset({'36', '53', '63'}),
 frozenset({'36', '53', '59'}),
 frozenset({'34', '36', '53'}),
 frozenset({'110', '24', '85'}),
 frozenset({'110', '24', '90'}),
 frozenset({'110', '24', '86'}),
 frozenset({'110', '24', '34'}),
 frozenset({'110', '24', '36'}),
 fr

In [25]:
for item in L[2]:
         if item.intersection('2'):
             print (item)

frozenset({'53', '28', '2'})
frozenset({'53', '39', '2'})
frozenset({'53', '90', '2'})
frozenset({'53', '86', '2'})
frozenset({'85', '53', '2'})
frozenset({'53', '34', '2'})
frozenset({'39', '28', '2'})
frozenset({'90', '28', '2'})
frozenset({'86', '28', '2'})
frozenset({'85', '28', '2'})
frozenset({'63', '28', '2'})
frozenset({'28', '2', '59'})
frozenset({'34', '28', '2'})
frozenset({'39', '2', '59'})
frozenset({'93', '86', '2'})
frozenset({'93', '63', '2'})
frozenset({'93', '2', '59'})
frozenset({'93', '34', '2'})
frozenset({'93', '39', '2'})
frozenset({'90', '93', '2'})
frozenset({'90', '86', '2'})
frozenset({'90', '63', '2'})
frozenset({'90', '2', '59'})
frozenset({'90', '2', '36'})
frozenset({'90', '34', '2'})
frozenset({'90', '2', '23'})
frozenset({'90', '39', '2'})
frozenset({'86', '2', '59'})
frozenset({'39', '86', '2'})
frozenset({'85', '93', '2'})
frozenset({'85', '90', '2'})
frozenset({'85', '86', '2'})
frozenset({'85', '76', '2'})
frozenset({'85', '67', '2'})
frozenset({'85